In [1]:
import os
import xlsxwriter
import mysql.connector as server
import json

moneda = "MXN"
todas_emisoras = {}

db = server.connect(host="192.168.100.200", user="mensajero", passwd="password", db="db_trading")
query = db.cursor()
query.execute(f"SELECT emisora, periodo FROM fundamentales")
lista_periodo_turnos = query.fetchall()

query.execute("SELECT * FROM emisoras")
todas_emisoras_bruto = query.fetchall()
db.close()

for x in todas_emisoras_bruto:
    todas_emisoras[x[1]] = x[3]

for turno in lista_periodo_turnos:
    clave_pizarra_turno = turno[0]
    periodo_turno = turno[1]
    nombre_emisora = todas_emisoras[clave_pizarra_turno]

    db = server.connect(host="192.168.100.200", user="mensajero", passwd="password", db="db_trading")
    query = db.cursor()
    query.execute(f"SELECT eeff FROM fundamentales WHERE periodo = '{periodo_turno}' AND emisora = '{clave_pizarra_turno}'")
    financieros = query.fetchall()
    financieros = json.loads(financieros[0][0])
    db.close()

    """Inicio de maquila Excel"""
    try:
        os.mkdir(f"{os.path.abspath('')}/{clave_pizarra_turno}")
    except FileExistsError:
        pass
    ruta = f"{os.path.abspath('')}/{clave_pizarra_turno}/{clave_pizarra_turno}_{periodo_turno}_EstadosFinancieros.xlsx"
    if os.path.exists(ruta) == True:
        pass
    else:
        workbook = xlsxwriter.Workbook(ruta)
        worksheet_balance = workbook.add_worksheet('Estado de Posicion Financiera')
        worksheet_resultados_ytd = workbook.add_worksheet('Estado de Resultados YTD')
        worksheet_resultados_trimestre = workbook.add_worksheet('Estado de Resultados Trimestre')
        worksheet_flujo = workbook.add_worksheet('Estado de Flujos de Efectivo')
        negritas_centrado = workbook.add_format({'bold':True})
        centrado = negritas_centrado.set_align('center')
        cashbaby = workbook.add_format({'num_format':'#,##0.00;[Red](#,##0.00)'})

        for ronda in range(2):

            if ronda == 0:
                columna_numeros = 2
            elif ronda == 1:
                columna_numeros = 3

            #Balance General
            try:
                worksheet_balance.write(0,0,nombre_emisora,negritas_centrado)
                worksheet_balance.write(0,2,f'Cifras en {moneda}',centrado)
                worksheet_balance.write(2,columna_numeros,str(list(financieros["posicion"].keys())[ronda]),negritas_centrado)

                for i, (llave, valor) in enumerate(financieros["posicion"][list(financieros["posicion"].keys())[ronda]].items()):
                    worksheet_balance.write(2+i,0,llave)
                    worksheet_balance.write(2+i,1,valor[0])
                    try:
                        worksheet_balance.write(2+i,columna_numeros,valor[1],cashbaby)
                    except IndexError:
                        pass
            except (IndexError, KeyError):
                pass


            #Estado de Resultados YTD y Trimestre
            try:
                worksheet_resultados_ytd.write(0,0,nombre_emisora, negritas_centrado)
                worksheet_resultados_ytd.write(0,2,f'Cifras en {moneda}',centrado)
                worksheet_resultados_ytd.write(2,columna_numeros,str(list(financieros["resultado_acumulado"].keys())[ronda]),negritas_centrado)

                for i, (llave, valor) in enumerate(financieros["resultado_acumulado"][list(financieros["resultado_acumulado"].keys())[ronda]].items()):
                    worksheet_resultados_ytd.write(2+i,0,llave)
                    worksheet_resultados_ytd.write(2+i,1,valor[0])
                    try:
                        worksheet_resultados_ytd.write(2+i,columna_numeros,valor[1],cashbaby)
                    except IndexError:
                        pass
            except (IndexError, KeyError):
                pass

            try:
                worksheet_resultados_trimestre.write(0,0,nombre_emisora, negritas_centrado)
                worksheet_resultados_trimestre.write(0,2,f'Cifras en {moneda}',centrado)
                worksheet_resultados_trimestre.write(2,columna_numeros,str(list(financieros["resultado_trimestre"].keys())[ronda]),negritas_centrado)

                for i, (llave, valor) in enumerate(financieros["resultado_trimestre"][list(financieros["resultado_trimestre"].keys())[ronda]].items()):
                    worksheet_resultados_trimestre.write(2+i,0,llave)
                    worksheet_resultados_trimestre.write(2+i,1,valor[0])
                    try:
                        worksheet_resultados_trimestre.write(2+i,columna_numeros,valor[1],cashbaby)
                    except IndexError:
                        pass
            except (IndexError, KeyError):
                pass

            #Flujo de Efectivo
            try:
                worksheet_flujo.write(0,0,nombre_emisora, negritas_centrado)
                worksheet_flujo.write(0,2,f'Cifras en {moneda}',centrado)
                worksheet_flujo.write(2,columna_numeros,str(list(financieros["flujos"].keys())[ronda]),negritas_centrado)

                for i, (llave, valor) in enumerate(financieros["flujos"][list(financieros["flujos"].keys())[ronda]].items()):
                    worksheet_flujo.write(2+i,0,llave)
                    worksheet_flujo.write(2+i,1,valor[0])
                    try:
                        worksheet_flujo.write(2+i,columna_numeros,valor[1],cashbaby)
                    except IndexError:
                        pass
            except (IndexError, KeyError):
                pass

            worksheet_balance.set_column('A:B',116)
            worksheet_balance.set_column('C:D',18)
            worksheet_resultados_ytd.set_column('A:B',68)
            worksheet_resultados_ytd.set_column('C:D',25)
            worksheet_resultados_trimestre.set_column('A:B',68)
            worksheet_resultados_trimestre.set_column('C:D',25)
            worksheet_flujo.set_column('A:B',108)
            worksheet_flujo.set_column('C:D',25)

        workbook.close()
        print(f"Procesado: {clave_pizarra_turno} {periodo_turno}")
print("Proceso terminado.")

Procesado: CERAMIC 4T_2021
Procesado: GCARSO 4T_2021
Procesado: GCC 4T_2021
Procesado: GMEXICO 4T_2021
Procesado: GMXT 4T_2021
Procesado: GSANBOR 4T_2021
Procesado: FIBRAPL 4T_2021
Procesado: FNOVA 4T_2021
Proceso terminado.
